<a href="https://colab.research.google.com/github/GArdennes/Research-Studies/blob/main/Standard_Microservice_Placement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import modules
from typing import List, Dict, Union
import time
import networkx as nx
from matplotlib import pyplot as plt
import numpy as np
import random
import pandas as pd

In [ ]:
#initialize variables and inputs
num_microservices = 4
microservices = [
    {"name": "microservice1", "ram": 512, "input": 256, "output": 256, "cpu": 50},
    {"name": "microservice2", "ram": 1024, "input": 512, "output": 512, "cpu": 100},
    {"name": "microservice3", "ram": 2048, "input": 1024, "output": 1024, "cpu": 200},
    {"name": "microservice4", "ram": 4096, "input": 2048, "output": 2048, "cpu": 400},
]

num_applications = 1

num_edge_nodes = 4
edge_nodes = [
    {"name": "edge_node1", "speed": 100, "ram": 2048, "uplink": 512, "downlink": 512, "busy_power": 100, "idle_power": 50, "cpu": 1000},
    {"name": "edge_node2", "speed": 200, "ram": 4096, "uplink": 1024, "downlink": 1024, "busy_power": 150, "idle_power": 75, "cpu": 2000},
    {"name": "edge_node3", "speed": 300, "ram": 8192, "uplink": 2048, "downlink": 2048, "busy_power": 200, "idle_power": 100, "cpu": 2500},
    #{"name": "edge_node4", "speed": 400, "ram": 16384, "uplink": 4096, "downlink": 4096, "busy_power": 250, "idle_power": 100, "cpu": 4000},
]

mappedNodes = []
mplaced = []
notPlacedPaths = []
TimeOutmicroservice = []
mapNodeToMicroservice = []

In [ ]:
def mapNodeTomicroservice(node, microservice, mappedNodes):
    """
    Maps a microservice to a node based on available resources and network latency
    """
    for n in mappedNodes:
        if n[0]['name'] == node['name']:
            n[1].append(microservice)
            return True
    mappedNodes.append((node, [microservice]))
    return True


In [ ]:
def has_enough_resources(node, microservice):
    if node['speed'] < microservice['cpu']:
        return False
    if node['ram'] < microservice['ram']:
        return False
    if node['uplink'] < microservice['input']:
        return False
    if node['downlink'] < microservice['output']:
        return False
    return True


In [ ]:
def get_power_usage(node: Dict[str, Union[int, float]]) -> float:
    """
    Computes the power usage of a given node.

    Parameters:
    node (dict): A dictionary containing the details of a node.

    Returns:
    float: The power usage of the node.
    """
    if node["status"] == "busy":
        return node["busy_power"]
    else:
        return node["idle_power"]


def allocate_resources(node: Dict[str, Union[int, float]], microservice: Dict[str, Union[int, float]]) -> None:
    """
    Allocates resources to a microservice on a given node.

    Parameters:
    node (dict): A dictionary containing the details of a node.
    microservice (dict): A dictionary containing the details of a microservice.

    Returns:
    None.
    """
    node["ram"] -= microservice["ram"]
    node["cpu"] -= microservice["cpu"]
    node["uplink"] -= microservice["input"]
    node["downlink"] -= microservice["output"]
    node["status"] = "busy"


In [ ]:
def place_microservice_on_node(node, microservice, mappedNodes, mplaced, notPlacedPaths):
    if has_enough_resources(node, microservice):
        mappedNodes[node]['resources'] = allocate_resources(mappedNodes[node]['resources'], microservice)
        mappedNodes[node]['microservices'].append(microservice['id'])
        mappedNodes[node]['used_power'] += get_power_usage(node, microservice)
        mplaced.append(microservice['id'])
    else:
        notPlacedPaths.append((microservice['id'], node['id']))


In [ ]:
#Get the Directed Acyclic Graph
graph = nx.DiGraph()
#graph = nx.MultiGraph()
graph.add_edges_from([("microservice1","microservice2"),("microservice2","microservice3"),("microservice2","microservice4"),("microservice4","microservice1")])

plt.tight_layout()
nx.draw_networkx(graph, arrows = True)
plt.savefig("g1.png",format="PNG")
plt.clf()

<Figure size 640x480 with 0 Axes>

In [ ]:
# Initialize timer
start_time = time.time()

# Set timeout
timeout = 120

# While there are microservices to place and the timeout has not been reached
while microservices and (time.time() - start_time < timeout):
    next_microservice = microservices[0]

    # Try to place the microservice on the next available node
    for next_node in edge_nodes:
        if has_enough_resources(next_node, next_microservice):
            allocate_resources(next_node, next_microservice)
            mapNodeTomicroservice(next_node, next_microservice, mappedNodes)
            mappedNodes.append((next_microservice, next_node))
            microservices.remove(next_microservice)
            mplaced.append(next_microservice)
            break

    # If the microservice could not be placed, add it to the list of not placed microservices
    if next_microservice in microservices:
        notPlacedPaths.append(next_microservice)
        microservices.remove(next_microservice)

    # Iterate over not placed paths
    while len(notPlacedPaths) > 0 and (time.time() - start_time < timeout):
        for ms in notPlacedPaths:
            if not has_enough_resources(next_node, ms):
                continue
            else:
                allocate_resources(next_node, ms)
                mapNodeTomicroservice(next_node, ms, mappedNodes)
                mplaced.append(ms)
                notPlacedPaths.remove(ms)

# Calculate time taken
elapsed_time = time.time() - start_time

# Print the results
print(f"Placed microservices: {mplaced}")
print(f"Not placed microservices: {notPlacedPaths}")
print(f"Elapsed time: {elapsed_time}")


Placed microservices: [{'name': 'microservice1', 'ram': 512, 'input': 256, 'output': 256, 'cpu': 50}, {'name': 'microservice2', 'ram': 1024, 'input': 512, 'output': 512, 'cpu': 100}, {'name': 'microservice3', 'ram': 2048, 'input': 1024, 'output': 1024, 'cpu': 200}]
Not placed microservices: [{'name': 'microservice4', 'ram': 4096, 'input': 2048, 'output': 2048, 'cpu': 400}]
Elapsed time: 120.00033736228943
